# CrewAI with Couchbase Short-Term Memory

This notebook demonstrates how to implement a custom storage backend for CrewAI's memory system using Couchbase and vector search. Here's a breakdown of each section:

How to run this tutorial
----------------------
This tutorial is available as a Jupyter Notebook (.ipynb file) that you can run 
interactively. You can access the original notebook here.

You can either:
- Download the notebook file and run it on [Google Colab](https://colab.research.google.com)
- Run it on your system by setting up the Python environment

Before you start
---------------

1. Create and Deploy Your Free Tier Operational cluster on [Capella](https://cloud.couchbase.com/sign-up)
   - To get started with [Couchbase Capella](https://cloud.couchbase.com), create an account and use it to deploy 
     a forever free tier operational cluster
   - This account provides you with an environment where you can explore and learn 
     about Capella with no time constraint
   - To learn more, please follow the [Getting Started Guide](https://docs.couchbase.com/cloud/get-started/create-account.html)

2. Couchbase Capella Configuration
   When running Couchbase using Capella, the following prerequisites need to be met:
   - Create the database credentials to access the required bucket (Read and Write) 
     used in the application
   - Allow access to the Cluster from the IP on which the application is running by following the [Network Security documentation](https://docs.couchbase.com/cloud/security/security.html#public-access)

# Memory in AI Agents
Memory in AI agents is a crucial capability that allows them to retain and utilize information across interactions, making them more effective and contextually aware. Without memory, agents would be limited to processing only the immediate input, lacking the ability to build upon past experiences or maintain continuity in conversations.

## Types of Memory in AI Agents

### Short-term Memory
- Retains recent interactions and context
- Typically spans the current conversation or session  
- Helps maintain coherence within a single interaction flow
- In CrewAI, this is what we're implementing with the Couchbase storage

### Long-term Memory
- Stores persistent knowledge across multiple sessions
- Enables agents to recall past interactions even after long periods
- Helps build cumulative knowledge about users, preferences, and past decisions

## How Memory Works in Agents
Memory in AI agents typically involves:
- Storage: Information is encoded and stored in a database (like Couchbase, ChromaDB, or other vector stores)
- Retrieval: Relevant memories are fetched based on semantic similarity to current context
- Integration: Retrieved memories are incorporated into the agent's reasoning process

In the CrewAI example, the CouchbaseStorage class implements:
- save(): Stores new memories with metadata
- search(): Retrieves relevant memories based on semantic similarity
- reset(): Clears stored memories when needed

## Benefits of Memory in AI Agents
- Contextual Understanding: Agents can refer to previous parts of a conversation
- Personalization: Remembering user preferences and past interactions
- Learning and Adaptation: Building knowledge over time to improve responses
- Task Continuity: Resuming complex tasks across multiple interactions
- Collaboration: In multi-agent systems like CrewAI, memory enables agents to build on each other's work

## Memory in CrewAI Specifically
In CrewAI, memory serves several important functions:
- Agent Specialization: Each agent can maintain its own memory relevant to its expertise
- Knowledge Transfer: Agents can share insights through memory when collaborating on tasks
- Process Continuity: In sequential processes, later agents can access the work of earlier agents
- Contextual Awareness: Agents can reference previous findings when making decisions

The vector-based approach (using embeddings) is particularly powerful because it allows for semantic search - finding memories that are conceptually related to the current context, not just exact keyword matches.

By implementing custom storage like Couchbase, you gain additional benefits like persistence, scalability, and the ability to leverage enterprise-grade database features for your agent memory systems.

## Install Required Libraries
This section installs the necessary Python packages:
- `crewai`: The main CrewAI framework
- `langchain-couchbase`: LangChain integration for Couchbase
- `langchain-openai`: LangChain integration for OpenAI
- `python-dotenv`: For loading environment variables

In [1]:
%pip install --quiet crewai langchain-couchbase langchain-openai python-dotenv

Note: you may need to restart the kernel to use updated packages.


## Importing Necessary Libraries
The script starts by importing a series of libraries required for various tasks, including handling JSON, logging, time tracking, Couchbase connections, embedding generation, and dataset loading.

In [2]:
import os
import logging
import uuid
import json
import time
from datetime import timedelta
from typing import Any, Dict, List, Optional
from dotenv import load_dotenv

# CrewAI imports
from crewai.memory.storage.rag_storage import RAGStorage
from crewai.memory.short_term.short_term_memory import ShortTermMemory
from crewai import Agent, Crew, Task, Process

# Couchbase imports
from couchbase.cluster import Cluster
from couchbase.options import ClusterOptions
from couchbase.auth import PasswordAuthenticator
from couchbase.diagnostics import PingState, ServiceType
from langchain_couchbase.vectorstores import CouchbaseVectorStore
from langchain_openai import OpenAIEmbeddings, ChatOpenAI

# Configure logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

## Loading Sensitive Information
In this section, we prompt the user to input essential configuration settings needed. These settings include sensitive information like database credentials, and specific configuration names. Instead of hardcoding these details into the script, we request the user to provide them at runtime, ensuring flexibility and security.

The script uses environment variables to store sensitive information, enhancing the overall security and maintainability of your code by avoiding hardcoded values.

In [3]:
load_dotenv()

# Verify environment variables
required_vars = ['OPENAI_API_KEY', 'CB_HOST', 'CB_USERNAME', 'CB_PASSWORD']
for var in required_vars:
    if not os.getenv(var):
        raise ValueError(f"{var} environment variable is required")

## Implement CouchbaseStorage

This section demonstrates the implementation of a custom vector storage solution using Couchbase:

In [4]:
class CouchbaseStorage(RAGStorage):
    def __init__(self, type: str, allow_reset: bool = True, embedder_config: Optional[Dict[str, Any]] = None, crew: Optional[Any] = None):
        super().__init__(type, allow_reset, embedder_config, crew)
        self._initialize_app()

    def search(self, query: str, limit: int = 3, filter: Optional[dict] = None, score_threshold: float = 0) -> List[Dict[str, Any]]:
        try:
            # Add type filter
            search_filter = {"memory_type": self.type}
            if filter:
                search_filter.update(filter)

            # Execute search
            results = self.vector_store.similarity_search_with_score(
                query,
                k=limit,
                filter=search_filter
            )
            
            # Format results and deduplicate by content
            seen_contents = set()
            formatted_results = []
            
            for i, (doc, score) in enumerate(results):
                if score >= score_threshold:
                    content = doc.page_content
                    if content not in seen_contents:
                        seen_contents.add(content)
                        formatted_results.append({
                            "id": doc.metadata.get("memory_id", str(i)),
                            "metadata": doc.metadata,
                            "context": content,
                            "score": float(score)
                        })
            return formatted_results
        except Exception as e:
            logger.error(f"Search failed: {str(e)}")
            return []

    def save(self, value: Any, metadata: Dict[str, Any]) -> None:
        try:
            # Generate unique ID
            memory_id = str(uuid.uuid4())
            timestamp = int(time.time() * 1000)
            
            # Prepare metadata
            if not metadata:
                metadata = {}
            metadata.update({
                "memory_id": memory_id,
                "memory_type": self.type,
                "timestamp": timestamp,
                "source": "crewai"
            })

            # Convert value to string if needed
            if isinstance(value, (dict, list)):
                value = json.dumps(value)
            elif not isinstance(value, str):
                value = str(value)

            # Save to vector store
            self.vector_store.add_texts(
                texts=[value],
                metadatas=[metadata],
                ids=[memory_id]
            )
        except Exception as e:
            logger.error(f"Save failed: {str(e)}")
            raise

    def reset(self) -> None:
        if not self.allow_reset:
            return

        try:
            self.cluster.query(
                f"DELETE FROM `{self.bucket_name}`.`{self.scope_name}`.`{self.collection_name}` WHERE memory_type = $type",
                type=self.type
            ).execute()
        except Exception as e:
            logger.error(f"Reset failed: {str(e)}")
            raise

    def _initialize_app(self):
        try:
            # Initialize embeddings
            if self.embedder_config and self.embedder_config.get("provider") == "openai":
                self.embeddings = OpenAIEmbeddings(
                    openai_api_key=os.getenv('OPENAI_API_KEY'),
                    model=self.embedder_config.get("config", {}).get("model", "text-embedding-3-small")
                )
            else:
                self.embeddings = OpenAIEmbeddings(
                    openai_api_key=os.getenv('OPENAI_API_KEY'),
                    model="text-embedding-3-small"
                )

            # Connect to Couchbase
            auth = PasswordAuthenticator(
                os.getenv('CB_USERNAME', ''),
                os.getenv('CB_PASSWORD', '')
            )
            options = ClusterOptions(auth)
            
            # Initialize cluster connection
            self.cluster = Cluster(os.getenv('CB_HOST', ''), options)
            self.cluster.wait_until_ready(timedelta(seconds=5))

            # Check search service
            ping_result = self.cluster.ping()
            search_available = False
            for service_type, endpoints in ping_result.endpoints.items():
                if service_type == ServiceType.Search:
                    for endpoint in endpoints:
                        if endpoint.state == PingState.OK:
                            search_available = True
                            break
                    break
            if not search_available:
                raise RuntimeError("Search/FTS service not found or not responding")
            
            # Set up storage configuration
            self.bucket_name = os.getenv('CB_BUCKET_NAME', 'vector-search-testing')
            self.scope_name = os.getenv('SCOPE_NAME', 'shared')
            self.collection_name = os.getenv('COLLECTION_NAME', 'crew')
            self.index_name = os.getenv('INDEX_NAME', 'vector_search_crew')

            # Initialize vector store
            self.vector_store = CouchbaseVectorStore(
                cluster=self.cluster,
                bucket_name=self.bucket_name,
                scope_name=self.scope_name,
                collection_name=self.collection_name,
                embedding=self.embeddings,
                index_name=self.index_name
            )
        except Exception as e:
            logger.error(f"Initialization failed: {str(e)}")
            raise

## Test Basic Storage

Test storing and retrieving a simple memory:

In [5]:
# Initialize storage
storage = CouchbaseStorage(
    type="short_term",
    embedder_config={
        "provider": "openai",
        "config": {"model": "text-embedding-3-small"}
    }
)

# Reset storage
storage.reset()

# Test storage
test_memory = "Pep Guardiola praised Manchester City's current form, saying 'The team is playing well, we are in a good moment. The way we are training, the way we are playing - I am really pleased.'"
test_metadata = {"category": "sports", "team": "manchester_city", "manager": "guardiola"}
storage.save(test_memory, test_metadata)

# Test search
results = storage.search("What did Guardiola say about Man City's form?", limit=1)
for result in results:
    print(f"Found: {result['context']}\nScore: {result['score']}\nMetadata: {result['metadata']}")

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"


Found: Pep Guardiola praised Manchester City's current form, saying 'The team is playing well, we are in a good moment. The way we are training, the way we are playing - I am really pleased.'
Score: 0.7201692461967468
Metadata: {}


## Test CrewAI Integration

Create agents and tasks to test memory retention:

In [6]:
# Initialize language model
llm = ChatOpenAI(model="gpt-4o", temperature=0.7)

# Create agents
researcher = Agent(
    role='Sports Analyst',
    goal='Research Manchester City news and updates',
    backstory='Expert at analyzing football news and manager statements',
    llm=llm,
    memory=True,
    memory_storage=ShortTermMemory(storage=storage)
)

writer = Agent(
    role='Sports Writer',
    goal='Create clear sports reports',
    backstory='Expert at writing engaging sports content',
    llm=llm,
    memory=True,
    memory_storage=ShortTermMemory(storage=storage)
)

# Create tasks
research_task = Task(
    description='Research Pep Guardiola\'s recent comments about Manchester City\'s form and performance.',
    agent=researcher,
    expected_output="A detailed analysis of Guardiola's statements about Man City's current form."
)

writing_task = Task(
    description='Write a summary of Guardiola\'s reaction to Manchester City\'s current form.',
    agent=writer,
    context=[research_task],
    expected_output="A clear report on Guardiola's assessment of Man City's form."
)

# Create crew
crew = Crew(
    agents=[researcher, writer],
    tasks=[research_task, writing_task],
    process=Process.sequential,
    memory_storage=ShortTermMemory(storage=storage),
    # memory=False,  # Change to False if handling memory at the agent level
    verbose=True
)

# Run crew
result = crew.kickoff()
print("\nCrew Result:\n", "-"*80)
print(result)

╭──────────────────────────────────────────── Crew Execution Started ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Started                                                                                         │
│  Name: crew                                                                                                     │
│  ID: 71b28c72-11f7-4f46-9f12-7187cbfa2de3                                                                       │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

🚀 Crew: crew
└── 📋 Task: 412aec4d-b525-488f-b711-b26e0ef31942
       Status: Executing Task...

🚀 Crew: crew
└── 📋 Task: 412aec4d-b525-488f-b711-b26e0ef31942
       Status: Executing Task...
    └── 🤖 Agent: Sports Analyst
            Status: In Progress

# Agent: Sports Analyst
## Task: Research Pep Guardiola's recent comments about Manchester City's form and performance.


🚀 Crew: crew
└── 📋 Task: 412aec4d-b525-488f-b711-b26e0ef31942
       Status: Executing Task...
    └── 🤖 Agent: Sports Analyst
            Status: In Progress
        └── 🧠 Thinking...

...

🚀 Crew: crew
├── 📋 Task: 412aec4d-b525-488f-b711-b26e0ef31942
│      Assigned to: Sports Analyst
│      Status: ✅ Completed
│   └── 🤖 Agent: Sports Analyst
│           Status: ✅ Completed
└── 📋 Task: 35e7e23c-e909-4719-93c9-3f75ba685a5f
       Status: Executing Task...
    └── 🤖 Agent: Sports Writer
            Status: ✅ Completed

🚀 Crew: crew
├── 📋 Task: 412aec4d-b525-488f-b711-b26e0ef31942
│      Assigned to: Sports Analyst
│      Status: ✅ Completed
│   └── 🤖 Agent: Sports Analyst
│           Status: ✅ Completed
└── 📋 Task: 35e7e23c-e909-4719-93c9-3f75ba685a5f
       Assigned to: Sports Writer
       Status: ✅ Completed
    └── 🤖 Agent: Sports Writer
            Status: ✅ Completed

╭──────────────────────────────────────────────── Task Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Task Completed                                                                                                 │
│  Name: 35e7e23c-e909-4719-93c9-3f75ba685a5f                                                                     │
│  Agent: Sports Writer                                                                                           │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────── Crew Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Completed                                                                                       │
│  Name: crew                                                                                                     │
│  ID: 71b28c72-11f7-4f46-9f12-7187cbfa2de3                                                                       │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯


Crew Result:
 --------------------------------------------------------------------------------
Pep Guardiola has addressed Manchester City's current form with a mixture of optimism and caution. In recent press conferences and interviews, he has acknowledged the impressive performances the team has delivered in certain matches, showcasing their potential to dominate the game. However, Guardiola has been candid about the need for greater consistency if they are to achieve their season objectives.

He pointed out specific areas for improvement, such as maintaining focus for the entire 90 minutes of play and being more clinical in front of goal. These are crucial factors for City to convert their dominance into tangible results consistently. Guardiola has expressed satisfaction with the squad's depth, praising the contributions from new signings and the positive impact they have had on the team dynamics. He emphasized the importance of properly integrating these players into Manchester Ci

## Test Memory Retention

Query the stored memories to verify retention:

In [7]:
# Wait for memories to be stored
time.sleep(2)

# Query memories
memory_results = storage.search(
    query="What was Pep Guardiola's reaction to Manchester City's current form?",
    limit=2,
    score_threshold=0.0
)

print("\nMemory Search Results:\n", "-"*80)
for result in memory_results:
    print(f"Context: {result['context']}")
    print(f"Score: {result['score']}")
    print(f"Metadata: {result['metadata']}")
    print("-"*80)

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"



Memory Search Results:
 --------------------------------------------------------------------------------
Context: Pep Guardiola praised Manchester City's current form, saying 'The team is playing well, we are in a good moment. The way we are training, the way we are playing - I am really pleased.'
Score: 0.7479899525642395
Metadata: {}
--------------------------------------------------------------------------------


## Conclusion: Benefits of Couchbase for AI Memory Systems

As demonstrated in this tutorial, Couchbase offers several advantages for implementing AI agent memory:

1. **Native Vector Search**: Couchbase provides built-in support for vector embeddings and similarity search, making it ideal for semantic retrieval of agent memories.

2. **Performance**: The distributed architecture of Couchbase ensures fast retrieval of relevant memories for real-time agent interactions, even as your data scales.

3. **Scalability**: As an enterprise-ready solution, Couchbase can grow with your AI application needs, from development to production deployment.

4. **Unified Platform**: Store both vector embeddings and structured data in the same database, eliminating the need for multiple specialized systems.

5. **Flexible Querying**: Combine vector similarity search with traditional queries and filters to retrieve precisely the memories your agents need.

6. **Enterprise Features**: Benefit from Couchbase's security, high availability, and disaster recovery capabilities for production AI systems.

7. **Cloud-Native**: With Couchbase Capella, you can deploy your vector store in the cloud with minimal operational overhead.

The implementation we've created demonstrates how to:
- Connect CrewAI's memory system to Couchbase
- Store and retrieve agent memories using vector similarity
- Maintain context across multi-agent workflows
- Filter memories based on metadata

By using Couchbase as your vector store for CrewAI agents, you benefit from a production-ready
database with comprehensive vector search capabilities, suitable for both development and
enterprise deployment.